In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import mlflow

/Users/christelleren/DSS/design/code-envs/python/py36_mlflow/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  after removing the cwd from sys.path.


In [0]:
project = dataiku.api_client().get_default_project()
managed_folder = project.get_managed_folder('8ip4v4B7')

with project.setup_mlflow(managed_folder=managed_folder) as mlflow:

    # Note: if you don't call this (i.e. when no experiment is specified), the default one is used
    mlflow.set_experiment("My first experiment")

    with mlflow.start_run(run_name="my_run"):
        # ...your MLflow code...
        mlflow.log_param("a", 1)
        mlflow.log_metric("b", 2)

        # This uses the regular MLflow APIs

In [0]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset("mydataset")
mydataset_df = mydataset.get_dataframe()